In [ ]:
from functools import reduce
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

# 환경 변수를 로드하고 API 키를 설정합니다.
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 언어 모델 초기화
#llm = ChatOpenAI(temperature=0)
llm = ChatOpenAI(model_name="gpt-4", temperature=0)
# 캐시 디렉토리 설정
#cache_dir = LocalFileStore("./.cache/practice/")

# 텍스트 분할 설정
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    #separator="\n",
    chunk_size=1000,
    chunk_overlap=100,
)

# 문서 로드 및 분할
folder_path = "../../data/job/"
file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]

# Map 함수: 각 파일을 로드하고 분할하여 문서 리스트를 반환합니다.
def load_and_split(file_path):
    loader = CSVLoader(file_path, encoding='utf8')
    return loader.load_and_split(text_splitter=splitter)

# Reduce 함수: 여러 문서 리스트를 하나의 리스트로 결합합니다.
def combine_docs(accumulated_docs, docs):
    accumulated_docs.extend(docs)
    return accumulated_docs

# MapReduce 적용: 문서 로드 및 분할, 문서 결합
mapped_docs = map(load_and_split, file_paths)
docs = reduce(combine_docs, mapped_docs, [])

# 임베딩과 캐싱 설정
embeddings = OpenAIEmbeddings()
#cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# 벡터 저장소 생성
vectorstore = FAISS.from_documents(docs, embeddings)

# 검색기 설정
retriever = vectorstore.as_retriever(search_kwargs={'k': 5})

# 프롬프트 설정
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a helpful job recommendation assistant. 
            Answer the question based on the context below. You recommend a user's job. And just print the job name. Don't print duplicated jobs.
            You provide the order of the job recommendations in the order of the highest similarity. 

            1. Determines the similarity of the user's resume to a saved resume file. 
            2. It recommends up to 3 jobs based on the similarity. 

            \n\n
            {context}",
            """
        ),
        ("human", "{question}"),
    ]
)

# 실행 체인 정의 및 결과 출력
chain = (
    {
        "context": retriever,
        "question":RunnablePassthrough(),
    }
    | prompt
    | llm
)

result = chain.invoke("기술스택은 python, sql, java, kubernates이고, 프로젝트는 학교웹사이트를 개발한 적이 있습니다. 데이터베이스 구축과 백엔드 개발을 담당했습니다. 추천 직업을 알려주세요.")
print(result)